In [27]:
import numpy as np
import math as m
import networkx as nx
import matplotlib.pyplot as plt

In [28]:
def get_stimulus(A, beta, reward):
    arg = beta*(reward - A)
    return m.tanh(arg)

In [29]:
def update_p(aspiration_level, beta, reward, prev_action, p):
    s = get_stimulus(aspiration_level, beta, reward)
    if prev_action == 0: #previous action = cooperation
        if s >= 0:
            return p + (1-p)*s
        else:
            return p + p*s
    else: #previous action = defect
        if s >= 0:
            return p - p*s
        else:
            return p - (1-p)*s

In [30]:
def misimplement_prob_update(eps, prob):
    return prob*(1-eps) + (1-prob)*eps

In [31]:
def get_payoff_matrix(b,c):
    return np.array([[b-c, -c],
                     [b, 0]])

In [32]:
def get_payoffs(p1, p2):
    return payoff_mat[p1][p2]

In [33]:
def create_ring_graph(N, k):
    return nx.watts_strogatz_graph(N, k, 0)


#pos = nx.circular_layout(G_small_ring)
#plt.figure(3,figsize=(20,20)) 
#nx.draw_networkx(G_small_ring, pos=pos, with_labels=False)


In [80]:
def simulate_game(G, payoff_mat, k, rounds, A, beta, eps):
    nodes = len(list(G.nodes))
    probas = np.array([0.8 for _ in range(nodes)])
    counts = np.zeros(rounds)
    for r in range(rounds):
        payoffs = np.zeros(nodes)
        actions = np.random.binomial(1, p = (1 - probas))
        #print(actions)
        for node in list(G.nodes):
            for neighbour in list(G.neighbors(node)):
                payoffs[node] += get_payoffs(actions[node], actions[neighbour])
            payoffs[node]/k
        for node in list(G.nodes):
            probas[node] = misimplement_prob_update(eps, update_p(A, beta, payoffs[node], actions[node], probas[node]))
        counts[r] = nodes - np.count_nonzero(actions)
    return counts/nodes

In [81]:
N = 100
k = 2
G = create_ring_graph(N, k)
A = 1
beta = 0.2
eps = 0.05
payoff_mat = get_payoff_matrix(6, 1)

In [85]:
coops_lvl = simulate_game(G, payoff_mat, k, 50, A, beta, eps)
    
"""avg_coops = np.zeros(50)
for i in range(50):
    for trial in coops_lvl:
        avg_coops[i] += trial[i]
    avg_coops = avg_coops/100"""

'avg_coops = np.zeros(50)\nfor i in range(50):\n    for trial in coops_lvl:\n        avg_coops[i] += trial[i]\n    avg_coops = avg_coops/100'

In [86]:
def plot_coops(coops_lvl, r):
    x = np.arange(r)
    plt.plot(x, coops_lvl)
    plt.show()

In [87]:
plot_coops(coops_lvl, 50)

In [88]:
lbls = np.random.choice(100, replace=False, size=100)
mapping = {i : lbls[i] for i in range(100)}
H = nx.relabel_nodes(G, mapping)